In [16]:
import pandas as pd
import boto3
import io
import datetime
import os

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

# Create a boto3 S3 client
s3 = boto3.client('s3',
                  aws_access_key_id= AWS_ACCESS_KEY_ID,
                  aws_secret_access_key= AWS_SECRET_ACCESS_KEY)


s3_uri = 's3://fraud-data-manuel/Fraud.csv'

# Parse the S3 URI into its components
bucket = s3_uri.split('/')[2]
key = '/'.join(s3_uri.split('/')[3:])


# Retrieve the contents of the CSV file from S3
response = s3.get_object(Bucket=bucket, Key=key)
contents = response['Body'].read().decode('utf-8')

df = pd.read_csv(io.StringIO(contents))

fraud-data-manuel
Fraud.csv


In [17]:
df = df.sample(n=200, replace=True)

df.drop(columns='isFraud', inplace=True)

In [18]:
today = datetime.datetime.today()
date_str = today.strftime('%Y-%m-%d')

In [12]:
csv_string = df.to_csv(index=False)

object_key = f'transaction_{date_str}_fraud.csv'
content_type = "text/csv"

# Upload the CSV string to S3
import pandas as pd
import boto3
import io
import datetime
import oss3.put_object(Bucket=bucket, Key=object_key, Body=csv_string, ContentType=content_type)

{'ResponseMetadata': {'RequestId': 'JP8GPJN12JN8F7AY',
  'HostId': 'Os8jZThMgwfD7C4eHVA8meq6j85vIOjyrDaCsYuxsQ5SoxyCPTMyKsA/wmTwU5HgHVGAgcHvDZg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Os8jZThMgwfD7C4eHVA8meq6j85vIOjyrDaCsYuxsQ5SoxyCPTMyKsA/wmTwU5HgHVGAgcHvDZg=',
   'x-amz-request-id': 'JP8GPJN12JN8F7AY',
   'date': 'Mon, 13 Mar 2023 18:51:11 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"7f66d551f8b345b36c350e754fa34097"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"7f66d551f8b345b36c350e754fa34097"',
 'ServerSideEncryption': 'AES256'}

In [19]:
import pandas as pd
import boto3
import io
import datetime
import os

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")


s3 = boto3.client('s3',
                  aws_access_key_id= AWS_ACCESS_KEY_ID,
                  aws_secret_access_key= AWS_SECRET_ACCESS_KEY)



def create_sample_data(s3_uri:str):
    bucket = s3_uri.split('/')[2]
    key = '/'.join(s3_uri.split('/')[3:])

    response = s3.get_object(Bucket=bucket, Key=key)
    contents = response['Body'].read().decode('utf-8')

    df = pd.read_csv(io.StringIO(contents))
    return df

def upload_sample_data(df, bucket:str):
    
    df = df.sample(n=200, replace=True)
    df.drop(columns='isFraud', inplace=True)

    today = datetime.datetime.today()
    date_str = today.strftime('%Y-%m-%d')
    
    csv_string = df.to_csv(index=False)
    object_key = f'transaction_{date_str}_fraud.csv'
    content_type = "text/csv"
    s3.put_object(Bucket=bucket, Key=object_key, Body=csv_string, ContentType=content_type)
    
df = create_sample_data('s3://fraud-data-manuel/Fraud.csv')
upload_sample_data(df, 'fraud-data-manuel')
